In [1]:
import cv2 as cv
import numpy as np
import dlib
import math

In [2]:
# face detector object
detectFace = dlib.get_frontal_face_detector()
# landmarks detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
def midpoint(pts1, pts2):
    x, y = pts1
    x1, y1 = pts2
    xOut = int((x + x1)/2)
    yOut = int((y1 + y)/2)

    return (xOut, yOut)

In [4]:
def eucaldainDistance(pts1, pts2):
    x, y = pts1
    x1, y1 = pts2
    eucaldainDist = math.sqrt((x1 - x) ** 2 + (y1 - y) ** 2)

    return eucaldainDist

In [5]:
def faceDetector(image, gray, Draw=True):
    cordFace1 = (0, 0)
    cordFace2 = (0, 0)
    # getting faces from face detector
    faces = detectFace(gray)

    face = None
    # looping through All the face detected.
    for face in faces:
        # getting coordinates of face.
        cordFace1 = (face.left(), face.top())
        cordFace2 = (face.right(), face.bottom())

        # draw rectangle if draw is True.
        if Draw == True:
            cv.rectangle(image, cordFace1, cordFace2, (0, 255,0), 2)
    return image, face

In [6]:
def faceLandmarkDetector(image, gray, face, Draw=True):
    # calling the landmarks predictor
    landmarks = predictor(gray, face)
    pointList = []
    # looping through each landmark
    for n in range(0, 68):
        point = (landmarks.part(n).x, landmarks.part(n).y)
        # getting x and y coordinates of each mark and adding into list.
        pointList.append(point)
        # draw if draw is True.
        if Draw == True:
            # draw circle on each landmark
            cv.circle(image, point, 3, (255,0,0), 1)
    return image, pointList

In [7]:
def blinkDetector(eyePoints):
    top = eyePoints[1:3]
    bottom = eyePoints[4:6]
    
    # finding the mid point of above points
    topMid = midpoint(top[0], top[1])
    bottomMid = midpoint(bottom[0], bottom[1])
    
    # getting the actual width and height eyes using eucaldainDistance function
    VerticalDistance = eucaldainDistance(topMid, bottomMid)
    HorizontalDistance = eucaldainDistance(eyePoints[0], eyePoints[3])
    
    if VerticalDistance == 0:
        return 0.0
    
    #if HorizontalDistance == 0:
       # return 0.0
    
    blinkRatio = (HorizontalDistance/VerticalDistance)
    
    return blinkRatio, topMid, bottomMid

In [8]:
def EyeTracking(image, gray, eyePoints):
    # getting dimensions of image
    dim = gray.shape
    # creating mask
    mask = np.zeros(dim, dtype=np.uint8)

    # converting eyePoints into Numpy arrays
    PollyPoints = np.array(eyePoints, dtype=np.int32)
    # Filling the Eyes portion with WHITE color
    cv.fillPoly(mask, [PollyPoints], 255)

    # Writing gray image where color is White in the mask using Bitwise and operator
    eyeImage = cv.bitwise_and(gray, gray, mask=mask)

    # getting the max and min points of eye in order to crop the eyes from Eye image
    maxX = max(eyePoints, key=lambda item: item[0])[0]
    minX = min(eyePoints, key=lambda item: item[0])[0]
    maxY = max(eyePoints, key=lambda item: item[1])[1]
    minY = min(eyePoints, key=lambda item: item[1])[1]

    # other than eye area will be black, making it white
    eyeImage[mask == 0] = 255

    # cropping the eye from eyeImage
    cropedEye = eyeImage[minY:maxY, minX:maxX]

    # getting width and height of cropedEye
    height, width = cropedEye.shape

    divPartHorizontal = int(width / 3)
    divPartVertical = int(height / 2)  # Dividing the eye vertically into upper and lower parts

    # applying the threshold to the eye
    ret, thresholdEye = cv.threshold(cropedEye, 100, 255, cv.THRESH_BINARY)

    # dividing the eye into parts
    rightPart = thresholdEye[0:height, 0:divPartHorizontal]
    centerPart = thresholdEye[0:height, divPartHorizontal:divPartHorizontal * 2]
    leftPart = thresholdEye[0:height, divPartHorizontal * 2:width]
    upperPart = thresholdEye[0:divPartVertical, 0:width]  # Upper part of the eye
    lowerPart = thresholdEye[divPartVertical:height, 0:width]  # Lower part of the eye

    # counting Black pixel in each part using numpy
    rightBlackPx = np.sum(rightPart == 0)
    centerBlackPx = np.sum(centerPart == 0)
    leftBlackPx = np.sum(leftPart == 0)
    upperBlackPx = np.sum(upperPart == 0)
    lowerBlackPx = np.sum(lowerPart == 0)
    
    pos, color = Position([rightBlackPx, centerBlackPx, leftBlackPx,upperBlackPx,lowerBlackPx])
    
    return mask, pos, color

In [9]:
YELLOW = (0, 247, 255)
MAGENTA = (255, 0, 242)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
LIGHT_CYAN = (255, 204, 0)

def Position(ValuesList):

    maxIndex = ValuesList.index(max(ValuesList))
    posEye = ''
    color = [WHITE, BLACK]
    if maxIndex == 0:
        posEye = "Right"
        color = [YELLOW, BLACK]
    elif maxIndex == 1:
        posEye = "Center"
        color = [BLACK, MAGENTA]
    elif maxIndex == 2:
        posEye = "Left"
        color = [LIGHT_CYAN, BLACK]
    elif maxIndex == 3:
        posEye = "Up"
        color = [LIGHT_CYAN, BLACK]
    elif maxIndex == 4:
        posEye = "Down"
        color = [BLACK, MAGENTA]
    else:
        posEye = "Eye Closed"
        color = [BLACK, WHITE]
    
    return posEye, color

In [10]:
camera = cv.VideoCapture(0)
font = cv.FONT_HERSHEY_SIMPLEX

while True:
    # getting frame from camera
    ret, frame = camera.read()
    if ret == False:
        break

    # converting frame into Gry image.
    grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calling the face detector funciton
    image, face = faceDetector(frame, grayFrame)
    
    if face is not None:
        # Use landmark detector function
        image, PointList = faceLandmarkDetector(frame, grayFrame, face, False)
        
        # Right Eye
        RightEyePoint = PointList[36:42]
        LeftEyePoint = PointList[42:48]
        
        #leftRatio, topMid, bottomMid = blinkDetector(LeftEyePoint)
        rightRatio, rTop, rBottom = blinkDetector(RightEyePoint)
        
        #blinkRatio = (leftRatio + rightRatio)/2
        
        if rightRatio > 8:
            cv.putText(image, f'Blink', (50,50), font, 1, (255, 9, 2), 2)
        
        # Draw rectangles around the eyes
        right_eye_rect = cv.boundingRect(np.array(RightEyePoint))
        left_eye_rect = cv.boundingRect(np.array(LeftEyePoint))
        
        cv.rectangle(image, (right_eye_rect[0], right_eye_rect[1]), 
                     (right_eye_rect[0] + right_eye_rect[2], right_eye_rect[1] + right_eye_rect[3]), 
                     (0, 255, 0), 2)
        
        cv.rectangle(image, (left_eye_rect[0], left_eye_rect[1]), 
                     (left_eye_rect[0] + left_eye_rect[2], left_eye_rect[1] + left_eye_rect[3]), 
                     (0, 255, 0), 2)
        
        mask, pos, color = EyeTracking(frame, grayFrame, RightEyePoint)
        maskleft, leftPos, leftColor = EyeTracking(frame, grayFrame, LeftEyePoint)
        
        cv.line(image, (30,90), (100,90), color[0], 30)
        
        cv.putText(image, f'{pos}',(35,95), font, 0.6, color[1], 2)
        
        cv.imshow('Frame', frame)
        
    else:
        cv.imshow('Frame', frame)

    key = cv.waitKey(1)

    # if q is pressed on keyboard: quit
    if key == ord('q'):
        break

camera.release()
cv.destroyAllWindows()

TypeError: cannot unpack non-iterable float object